# Rationale
Let's train a RealPD-only network first, but I hope that there is an increase in metrics if we use both datasets together

In [1]:
import tensorflow as tf
import tf_mapper
from importlib import reload
reload(tf_mapper)
from tf_mapper import get_batched_dataset
# tf.enable_eager_execution()
# Helperfunctions to make your feature definition more readable

def read_tfrecord(example):
    features = { \
               'watch_accel':tf.io.FixedLenFeature([1500*3], tf.float32,),\
               'watch_gyro':tf.io.FixedLenFeature([1500*3], tf.float32,),\
               'watch_phone': tf.io.FixedLenFeature([1500*3], tf.float32,),\
               'has_watch_accel': tf.io.FixedLenFeature([1], tf.int64,),\
               'has_gyro':tf.io.FixedLenFeature([1], tf.int64,),\
               'has_phone': tf.io.FixedLenFeature([1], tf.int64,),\
               'on_off': tf.io.FixedLenFeature([1], tf.int64,),\
               'dyskinesia':tf.io.FixedLenFeature([1], tf.int64,),\
               'measurement_id': tf.io.FixedLenFeature([1], tf.int64,),\
               'tremor': tf.io.FixedLenFeature([1], tf.int64,),\
               'subjects': tf.io.FixedLenFeature([1], tf.int64,)
              }

    example = tf.io.parse_single_example(example, features)
    return example
def map_example_to_simple(example):
    data = example['data']
    data = tf.reshape(data, (1500,3))
    return data, (example['on_off'][0],)

/home/ms994/miniconda3/envs/keras-redo/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ms994/miniconda3/envs/keras-redo/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ms994/miniconda3/envs/keras-redo/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ms994/miniconda3/envs/keras-

In [2]:
def tf_is_in_set(a, b):
    return tf.reduce_sum(tf.cast(tf.equal(b, a), tf.int64)) >= 1

In [ ]:

def get_batched_dataset(filenames, batch_size, m_ids, max_queue_size=10,  n_process=4, is_train=False):
    option_no_order = tf.data.Options()
    option_no_order.experimental_deterministic = False

    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.with_options(option_no_order)
    dataset = dataset.interleave(tf.data.TFRecordDataset, cycle_length=1, num_parallel_calls=1)

    dataset = dataset.map(read_tfrecord, num_parallel_calls=n_process)
    dataset = dataset.filter(lambda example: tf_is_in_set(example["measurement_id"][0], tf.constant(m_ids, dtype=tf.int64)))
    dataset = dataset.map(map_example_to_simple)
    dataset = dataset.filter(lambda x, y: tf.not_equal(y[0], -1))
#     dataset = dataset.filter(lambda x, y: tf.math.reduce_std(x) > 0.05)
#     dataset = dataset.filter(lambda x, y: tf.not_equal(y[1], -1))
#     dataset = dataset.filter(lambda x, y: tf.not_equal(y[2], -1))

    
    dataset = dataset.repeat()
    if is_train:
        dataset = dataset.shuffle(2056)
#     dataset = dataset.batch(batch_size, drop_remainder=True)
    if is_train:
        dataset = dataset.prefetch(max_queue_size)
    else:
        dataset = dataset.prefetch(int(max_queue_size/4)) #store a lot less for the other sets to avoid wasting memory

    return dataset